In [ ]:
# Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
# 체인을 수동으로 구현해야 합니다.
# 체인에 ConversationBufferMemory를 부여합니다.
# 이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
# 체인에 다음 질문을 합니다:
    # Aaronson 은 유죄인가요?
    # 그가 테이블에 어떤 메시지를 썼나요?
    # Julia 는 누구인가요?


from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableMap

llm = ChatOpenAI(
   temperature=0.1,
#    streaming=True,
#    callbacks=[StreamingStdOutCallbackHandler()],
)

memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="history",
    return_messages=True,
)

loader = UnstructuredFileLoader("./files/document.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
    # length_function=len,
)

embeddings = OpenAIEmbeddings()
cache_dir = LocalFileStore("./cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings,cache_dir,)


docs = loader.load_and_split(text_splitter=splitter)
# print(docs)

vector_store = FAISS.from_documents(docs, cached_embeddings)

retriever = vector_store.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful assistant. Answer questions using only the following context.If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        # MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

# 메모리 로드 함수 정의
def load_memory(input):
    print(input)
    return memory.load_memory_variables({})['history']

# 체인 설정
chain = {"context": retriever,"question": RunnablePassthrough(),} | prompt | llm

def invoke_chain(question):
    result = chain.invoke({
        "question": question,
    })
    memory.save_context(
        {"input":question},
        {"output":result.content}
    )
    print(result)


content='He wrote "2+2=5" in the dust on the table.'


In [59]:
result = chain.invoke("Is Aaronson guilty?")
print(result)
result = chain.invoke("What message did he write in the table?")
print(result)
result = chain.invoke("Who is Julia?")
print(result)




content='According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'
content='He wrote "2+2=5" in the dust on the table.'
content='Julia is a character mentioned in the text. She is someone who Winston loves and cares for deeply, and he has a strong emotional connection to her.'
